<a href="https://colab.research.google.com/github/ThuanyDeSouza/N3_Security/blob/main/RSA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [54]:
# Encripta
import base64

def encrypt(public_key, message):
  n, e = public_key
  encrypted = []
  for msg in message:
    encodedChunk = pow(msg, int(e), int(n))
    encrypted.append(encodedChunk)

  return encrypted

def read_keyfile(keyfile):
  with open(keyfile, 'r') as file:
    lines = []
    for line in file:
      lines.append(line)

    return lines

def base64_to_int_chunk(base64_string, n):
  chunks = [base64_string[i:i+n] for i in range(0, len(base64_string), n)]
  converted = []
  result = 0
  for chunk in chunks:
    for c in reversed(chunk):
      result = result * 256
      result += ord(c)
    converted.append(result)
    result = 0

  return converted

def block_size(n: int) -> int:
  bsize = 0
  temp = n - 1

  while temp > 1:
    temp = int(temp / 2)
    bsize += 1

  return int(bsize/8)

def encrypt_file(keyfile, inputfile, outputfile):
  # Lê o arquivo a ser encriptado
  with open(inputfile, 'r') as file:
      data = file.read()
  # Converte o texto para base64
  data = base64.b64encode(data.encode()).decode()
  # Lê a chave
  public_key = read_keyfile(keyfile)
  # Retorna blocksize de acordo com n
  bs = block_size(int(public_key[0]))
  # Divide o texto de acordo com n e Converte para números longos
  test = base64_to_int_chunk(data, bs)
  # Encriptar dados
  encrypted = encrypt(public_key, test)
  # Escreve o arquivo encriptado
  with open(outputfile, 'w') as file:
    file.write('\n'.join(str(x) for x in encrypted))
  print("Arquivo encriptografado com sucesso!")

if __name__ == '__main__':
  keyfile = input("Qual o arquivo da chave publica? ")
  inputfile = input("Qual o arquivo da mensagem a ser criptografada? ")
  outputfile = input("Em qual arquivo gerar a mensagem criptografada? ")

  encrypt_file(keyfile, inputfile, outputfile)


Qual o arquivo da chave publica? public.txt
Qual o arquivo da mensagem a ser criptografada? testIn.txt
Em qual arquivo gerar a mensagem criptografada? testOut.txt
Arquivo encriptografado com sucesso!


In [55]:
# Decripta
import base64

def decrypt(private_key, message):
  n, d = private_key
  decrypted = []
  for msg in message:
    encodedChunk = pow(int(msg), int(d), int(n))
    decrypted.append(encodedChunk)

  return decrypted

def read_keyfile(keyfile):
  with open(keyfile, 'r') as file:
    lines = []
    for line in file:
      lines.append(line)

  return lines

def int_to_str(n: int) -> str:
  if n == 0:
    string_val = "0"
  else:
    string_val = ""
    while n > 0:
      quot, rem = divmod(n, 256)
      char_num = rem
      string_val += chr(char_num)
      n = quot

  return str(string_val)

def chunk_int_to_base64(int_chunks):
  base64 = ""
  for chunk in int_chunks:
    base64 += int_to_str(chunk)

  return base64

def decrypt_file(keyfile, inputfile, outputfile):
  # Lê o arquivo a ser descriptografado
  with open(inputfile, 'r') as file:
    lines = []
    for line in file:
      lines.append(line)
  # Lê a chave
  private_key = read_keyfile(keyfile)
  # Descriptografa os dados
  decrypted = decrypt(private_key, lines)
  # Converte de inteiro para string
  base64Text = chunk_int_to_base64(decrypted)
  # Converte o base64 para texto
  decoded_bytes = base64.b64decode(base64Text)
  decoded_string = decoded_bytes.decode('utf-8')
  # Escreve o arquivo encriptado
  with open(outputfile, 'w') as file:
    file.write(decoded_string)
  print("Arquivo descriptografado com sucesso!")

if __name__ == '__main__':
  keyfile = input("Qual o arquivo da chave privada? ")
  inputfile = input("Qual o arquivo da mensagem a ser descriptografada? ")
  outputfile = input("Em qual arquivo gerar a mensagem descriptografada? ")

  decrypt_file(keyfile, inputfile, outputfile)

Qual o arquivo da chave privada? private.txt
Qual o arquivo da mensagem a ser descriptografada? testOut.txt
Em qual arquivo gerar a mensagem descriptografada? testIn.txt
Arquivo descriptografado com sucesso!


In [ ]:
# Gerando as chaves
import linecache
import random
import math

# Função para gerar um número primo grande aleatório.
def generate_large_prime(file_path):
    line_count = sum(1 for line in open(file_path))
    random_line_numbers = random.sample(range(1, line_count + 1), 2)
    random_lines = [linecache.getline(file_path, line_number).strip() for line_number in random_line_numbers]
    return random_lines

# Função para verificar se um número é primo.
def is_prime(num):
    if num < 2:
        return False
    for i in range(2, int(math.sqrt(num)) + 1):
        if num % i == 0:
            return False
    return True

# Função para calcular o maior divisor comum entre dois números.
def gcd(a, b):
    while b != 0:
        a, b = b, a % b
    return a

#  Função para calcular o inverso modular de 'a' mod 'm'.
def mod_inverse(a, m):
    if gcd(a, m) != 1:
        return None
    u1, u2, u3 = 1, 0, a
    v1, v2, v3 = 0, 1, m

    while v3 != 0:
        q = u3 // v3
        v1, v2, v3, u1, u2, u3 = (
            u1 - q * v1,
            u2 - q * v2,
            u3 - q * v3,
            v1,
            v2,
            v3,
        )

    return u1 % m

# Função para gerar as chaves RSA.
def generate_rsa_keys():
    p, q = generate_large_prime("primeList.txt")
    p = int(p)
    q = int(q)
    n = p * q
    totient = (p - 1) * (q - 1)

    while True:
        e = random.randint(2, totient)
        if gcd(e, totient) == 1:
            break

    d = mod_inverse(e, totient)

    return (n, e), (n, d)

public_key, private_key = generate_rsa_keys()
print("Chave pública:", public_key)
print("Chave privada:", private_key)
# Escreve a chave publica
with open("public.txt", 'w') as file:
    file.write('\n'.join(str(x) for x in public_key))

# Escreve a chave privada
with open("private.txt", 'w') as file:
    file.write('\n'.join(str(x) for x in private_key))


Chave pública: (2314320848292068411, 472076984491637867)
Chave privada: (2314320848292068411, 1615980125760939203)
